In [ ]:
import ee
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

Getting Credentials and initializing Project

In [ ]:
from google.colab import userdata
key_data = userdata.get('SERVICE_ACCOUNT_CREDENTIALS')

credentials = ee.ServiceAccountCredentials(email = 'datascience@datascience-474615.iam.gserviceaccount.com',key_data=key_data)

ee.Initialize(credentials,
              project='datascience-474615',
              opt_url='https://earthengine-highvolume.googleapis.com'
)


In [ ]:
amazon_shape = ee.FeatureCollection('projects/datascience-474615/assets/amazonia_polygons')

In [ ]:
canopy_dataset = ee.ImageCollection("MODIS/061/MOD44B").select("Percent_Tree_Cover")

canopy_nominal_scale = canopy_dataset.first().projection().nominalScale().getInfo()

canopy_threshold = 50

Wir nutzen eine Baumkronenabdeckungs von 50% als unseren Schwellwert, den wir nutzen um eine gegebene Fläche als Regenwald zu klassifizieren. Dies basiert einer klassifizierung, die sich in der Literatur wiederfindet, wonach eine Abdeckung von 41% bis 70% als einen mittel dichten Wald klassifiziert wird.

https://www.sciencedirect.com/science/article/pii/S1470160X24011671

In [ ]:
import geemap

# Create a map object.
m = geemap.Map()
m.centerObject(amazon_shape,zoom = 4)

# Add the elevation model to the map object.
m.add_ee_layer(canopy_dataset.first().clip(amazon_shape).gte(canopy_threshold),{'palette': ['222222', '3faa00']},"2000")
m.add_ee_layer(canopy_dataset.sort('system:time_start', False).first().clip(amazon_shape).gte(canopy_threshold),{'palette': ['222222', '3faa00']},"Today")

# Display the map.
display(m)

In [ ]:
import datetime

def sum_img(img: ee.Image):
  sum = img.gte(canopy_threshold).reduceRegion(
      reducer = ee.Reducer.sum(),
      geometry = amazon_shape.bounds(),
      scale = canopy_nominal_scale,
      maxPixels = 1e9
  ).get('Percent_Tree_Cover')

  return ee.Feature(None,{
      'canopy' : ee.Number(sum).divide(10000/(canopy_nominal_scale*canopy_nominal_scale)).divide(1000000),
      'date' : img.get('system:time_start')
  })

canopy_feature_collection = canopy_dataset.map(sum_img)

canopy_cover_y = canopy_feature_collection.aggregate_array('canopy').getInfo()
canopy_cover_x = [datetime.date.fromtimestamp(int(x)/1000) for x in canopy_feature_collection.aggregate_array('date').getInfo()]

print(canopy_cover_y)
print(canopy_cover_x)



In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,4))

ax.grid(linestyle = 'dotted')
ax.plot(canopy_cover_x,canopy_cover_y)

ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.xaxis.set_label_text('Year')
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

ax.yaxis.set_units("Mha")
ax.yaxis.set_label_text('Rainforest Area (Megahektar)')

plt.show()

In [ ]:
import pandas as pd
import datetime

# needed for constant spacing between datestamps to allow for forecasts
canopy_cover_x_regular = [datetime.date(e.year, e.month, 1) for e in canopy_cover_x]

ts = pd.Series(canopy_cover_y, pd.DatetimeIndex(canopy_cover_x_regular,freq = pd.infer_freq(canopy_cover_x_regular)))

print(ts)

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(ts)
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

Der P-Wert ist über dem Signifikanzniveau von 0.05 und damit hat die Kurve einen Trend. Darunter können Sie außerdem die Kurve noch einmal sehen, nachdem eine einfache Differenzierung angewendet wurde.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,4))

ax.grid(linestyle = 'dotted')
ax.plot(ts.diff().dropna()) # Differencing Done here

ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_minor_locator(mdates.MonthLocator())
ax.xaxis.set_label_text('Year')
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(ts.diff().dropna(), lags=10)
plot_pacf(ts.diff().dropna(), lags=10)
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(
    ts, model='additive', extrapolate_trend='freq')

result.plot()
plt.tight_layout()
plt.show()

Dies zeigt, das wir keine (signifikante) Saisonaliät in unserer Zeitreihe haben und diesen Teil des SARIMA modells ignorieren können.

In [ ]:
!pip3 install pmdarima

In [ ]:
from pmdarima import auto_arima
auto_arima_model = auto_arima(ts,
                           start_p=0, start_q=0,
                           max_p=6, max_q=6,
                           d=1, max_d=2,trace=True,
                           error_action='ignore',
                           suppress_warnings=True,
                           stepwise=False)

In [ ]:
auto_arima_model.summary()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

differencing = int(auto_arima_model.order[1])

# exclude d first elements, due to the differencing producing sudden jumps in the model
in_sample_prediction = auto_arima_model.predict_in_sample()[differencing:]
out_of_sample_prediction = auto_arima_model.predict(n_periods=15)

fig, ax = plt.subplots(1, 1, figsize=(18,3))
ax.grid(linestyle = 'dotted')

ax.plot(ts, label='Observed-GEE')

# Statista rainforest loss: https://de.statista.com/statistik/daten/studie/1269192/umfrage/verlust-an-regenwald-brasilien/
loss = [1.62, 1.57, 2.02, 1.82, 1.42, 1.15, 1.08, 0.7, 1.15, 0.8, 1.12, 0.63, 0.94, 0.83, 2.83, 2.13, 1.35, 1.36, 1.7, 1.55, 1.77, 1.14, 2.82]

# numbers right here vary a lot, especially for the whole amazone basin, they range between 500 and 600 megahektar
area_2000 = 590

actual_area = [area_2000-sum(loss[:i+1]) for i in range(len(loss))]

ax.plot(pd.Series([area_2000,area_2000] + actual_area,index = ts.index), label = 'Observed-Statista')

ax.plot(pd.concat([in_sample_prediction,out_of_sample_prediction]), label='Arima Prediction and Forecast', linestyle='--')

ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_label_text('Year')

for label in ax.get_xticklabels(which='major'):
    label.set(rotation=45, horizontalalignment='right')

ax.yaxis.set_units("Mha")
ax.yaxis.set_label_text('Rainforest Area (Megahektar)')

ax.legend()
fig.show()

Hier ist noch eine Visuelle Darstellung der verlorenen/gewonnen Regenwaldfläche




In [ ]:
import geemap

m = geemap.Map()
m.centerObject(amazon_shape,zoom = 4)

m.add_ee_layer(canopy_dataset.first().gte(canopy_threshold).subtract(canopy_dataset.sort('system:time_start', False).first().gte(canopy_threshold)).clip(amazon_shape),{'min': -1, 'max':1, 'palette' : ['0f0','555','f00']},"Loss/Gain since 2000")

display(m)